In [1]:
import torch
import random
import copy
import random
import math
import os
import time
import torch.nn as nn
import torch.optim as optim
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a good puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
#print(tokenized_text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a','good', 'puppet', '##eer', '[SEP]']


# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,1]
print((indexed_tokens))


# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 103, 2001, 1037, 2204, 13997, 11510, 102]


In [4]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=

In [5]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

In [6]:
print(encoded_layers[-1])

tensor([[[-4.2205e-01,  2.1602e-01, -5.7724e-01,  ..., -6.3580e-01,
           5.0056e-01,  6.5595e-01],
         [-1.2114e+00,  6.0209e-02, -4.2991e-02,  ...,  8.4008e-03,
          -4.3899e-02, -3.3080e-01],
         [ 8.9214e-02, -4.4234e-01, -4.3575e-01,  ..., -5.3004e-03,
           5.2706e-01,  4.1737e-01],
         ...,
         [ 2.5614e-01,  2.2358e-01,  2.6866e-01,  ..., -5.3113e-02,
           5.8927e-01,  4.4406e-02],
         [-1.9936e-01, -5.3880e-04,  3.9020e-01,  ...,  5.8705e-01,
           1.5201e-01, -3.3313e-01],
         [ 8.3456e-01,  2.4024e-01, -2.8192e-01,  ...,  2.3706e-01,
          -3.3764e-01, -1.9108e-01]]])


In [7]:
print(encoded_layers[-1].shape)

torch.Size([1, 15, 768])


In [8]:
class First_Model(nn.Module):
    def __init__(self, input_dim, label_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.dropout = dropout
        
        self.fc = nn.Linear(input_dim, label_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        predictions = self.fc(src)
        
        return  predictions

In [9]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        input_feature = batch.text
        counterfact = copy.deepcopy(src)
        
        optimizer.zero_grad()
        
        output, predictions = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion1(output, trg) #+ criterion2(predictions, batch.label)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
class FareTEDClassifier(BertPreTrainedModel):
    def __init__(self, config, extra_concatenated_input_dim, ,num_labels=14, output_attentions=False, keep_multihead_output=False):
        
        super(FareTEDClassifier, self).__init__(config)
        self.output_attentions = output_attentions
        self.num_labels = num_labels
        self.bert = BertModel(config, output_attentions=output_attentions,
                                      keep_multihead_output=keep_multihead_output)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size+extra_concatenated_input_dim, num_labels)
        self.apply(self.init_bert_weights)#No Idea
        
    def forward(self, input_ids, extra_input ,token_type_ids=None, attention_mask=None, labels=None, head_mask=None):
        
        #Get the sentence representation from
        outputs = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False, head_mask=head_mask)
        if self.output_attentions:
            all_attentions, _, pooled_output = outputs
        else:
            _, pooled_output = outputs
        pooled_output = self.dropout(pooled_output)
        
        #here concatenate pooled_output and extra input
        concatenated_input = 
        logits = self.classifier(concatenated_input)
        return logits
#         if labels is not None:
            
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             return loss
#         elif self.output_attentions:
#             return all_attentions, logits
#         return logits


In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (labeled_batch,counterfactual_batch) in enumerate(iterator):
        
        
        
#         input_feature = batch.text
#         counterfact = copy.deepcopy(src)
        
#         optimizer.zero_grad()
        
#         output, predictions = model(src, trg)
        
#         #trg = [trg sent len, batch size]
#         #output = [trg sent len, batch size, output dim]
        
#         output = output[1:].view(-1, output.shape[-1])
#         trg = trg[1:].view(-1)
        
#         #trg = [(trg sent len - 1) * batch size]
#         #output = [(trg sent len - 1) * batch size, output dim]
        
#         loss = criterion1(output, trg) #+ criterion2(predictions, batch.label)
        
#         loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
#         optimizer.step()
        
#         epoch_loss += loss.item()


        
        
    return epoch_loss / len(iterator)